In [108]:
from pysmt.shortcuts import *  # Import all shortcuts from PySMT for creating logical expressions.
from pysmt.typing import BVType  # Import the bit-vector type for defining variables.

# Define the bit-vector size for our model.
n = 10  # This specifies that we will work with 10-bit vectors.

# Function to declare state variables for each state index i
def declare(i):
    state = {}  # Create a dictionary to hold the state variables.
    # Define each state variable as a bit-vector of size n, appending the index to make them unique.
    state['pc'] = Symbol('pc' + str(i), BVType(n))  # Program counter (pc) indicating current state.
    state['x'] = Symbol('x' + str(i), BVType(n))    # First multiplicand.
    state['y'] = Symbol('y' + str(i), BVType(n))    # Second multiplicand.
    state['a'] = Symbol('a' + str(i), BVType(n))    # Store initial value of x.
    state['b'] = Symbol('b' + str(i), BVType(n))    # Store initial value of y.
    state['z'] = Symbol('z' + str(i), BVType(n))    # Accumulated result of the multiplication.
    return state  # Return the dictionary of state variables.

# Initialize state variables, setting a and b to fixed values
def init(state, x_val, y_val):
    # Create conditions for the initial state.
    PC = Equals(state['pc'], BV(0, n))  # Set pc to 0 (initial state).
    X = Equals(state['x'], BV(x_val, n))  # Initialize x to x_val.
    Y = Equals(state['y'], BV(y_val, n))  # Initialize y to y_val.
    # Initialize a and check if it's less than 2^(n/2).
    A = And(Equals(state['a'], BV(x_val, n)), BVULT(BV(x_val, n), BV(2**(n//2), n)))
    # Initialize b and check if it's less than 2^(n/2).
    B = And(Equals(state['b'], BV(y_val, n)), BVULT(BV(y_val, n), BV(2**(n//2), n)))
    Z = Equals(state['z'], BV(0, n))  # Initialize z to 0.
    # Combine all initial conditions into a single logical expression.
    return And(PC, X, Y, A, B, Z)

# Transition conditions to simulate multiplication process
def trans(curr, prox):
    # Initial state transition from state 0 to 1
    t01 = And(
        Equals(curr['pc'], BV(0, n)),       # Current pc == 0
        Equals(prox['pc'], BV(1, n)),       # Next pc is set to 1
        Equals(prox['x'], curr['x']),       # x remains constant
        Equals(prox['y'], curr['y']),       # y remains constant
        Equals(prox['a'], curr['a']),       # a remains constant
        Equals(prox['b'], curr['b']),       # b remains constant
        Equals(prox['z'], BV(0, n))         # z is reset to 0
    )

    # Transition for ongoing multiplication when pc == 1
    t12 = And(
        Equals(curr['pc'], BV(1, n)),       # Current pc == 1 (in multiplication)
        NotEquals(curr['y'], BV(0, n)),     # Ensure y is not zero
        Equals(prox['x'], curr['x']),       # x remains constant
        Equals(prox['y'], BVSub(curr['y'], BV(1, n))), # Decrement y by 1
        Equals(prox['a'], curr['a']),       # a remains constant
        Equals(prox['b'], curr['b']),       # b remains constant
        Equals(prox['z'], BVAdd(curr['z'], curr['x'])), # Add x to z
        Equals(prox['pc'], BV(1, n))        # Stay in state 1
    )

    # Transition to the end of multiplication when y == 0
    t23 = And(
        Equals(curr['pc'], BV(1, n)),       # Current pc == 1
        Equals(curr['y'], BV(0, n)),        # y should be zero to transition
        Equals(prox['x'], curr['x']),       # x remains constant
        Equals(prox['y'], curr['y']),       # y remains constant
        Equals(prox['a'], curr['a']),       # a remains constant
        Equals(prox['b'], curr['b']),       # b remains constant
        Equals(prox['z'], curr['z']),       # z retains the final result
        Equals(prox['pc'], BV(2, n))        # Move to final state (pc == 2)
    )

    # Final state transition (loop)
    t33 = And(
        Equals(curr['pc'], BV(2, n)),       # Current pc == 2
        Equals(prox['pc'], BV(2, n)),       # Remain in final state
        Equals(prox['x'], curr['x']),       # x remains constant
        Equals(prox['y'], curr['y']),       # y remains constant
        Equals(prox['a'], curr['a']),       # a remains constant
        Equals(prox['b'], curr['b']),       # b remains constant
        Equals(prox['z'], curr['z'])        # z remains constant
    )

    # Return the logical expression representing any of the transitions
    return Or(t01, t12, t23, t33)

# Invariant that x * y + z = a * b
def invariant_check(state):
    # Check that the invariant holds true
    return Equals(
        BVAdd(BVMul(state['x'], state['y']), state['z']),  # x * y + z
        BVMul(state['a'], state['b'])                       # should equal a * b
    )

# Overflow condition
def overflow(state):
    # Define maximum value for n-bit vector
    max_val = BV((2**n) - 1, n)
    # Check if any state variable exceeds the maximum allowed value
    return Or(
        BVUGT(state['x'], max_val),
        BVUGT(state['y'], max_val),
        BVUGT(state['a'], max_val),
        BVUGT(state['b'], max_val),
        BVUGT(state['z'], max_val)
    )

# Bounded Model Checker function
def bmc_always(inv, K, x_val, y_val):
    with Solver() as solver:  # Create a solver instance
        # Declare states and add initial conditions for state 0
        states = [declare(i) for i in range(K + 1)]  # Create K + 1 states
        solver.add_assertion(init(states[0], x_val, y_val))  # Add initial conditions

        for k in range(K):
            # Add transition conditions between current and next state
            if k > 0:
                solver.add_assertion(trans(states[k - 1], states[k]))  # Add transition condition

            # Check if the invariant holds at each state
            solver.push()  # Push current context onto the stack
            solver.add_assertion(Not(inv(states[k])))  # Assert that the invariant does not hold

            # Check for overflow condition at each state
            solver.add_assertion(overflow(states[k]))  # Assert overflow condition

            if solver.solve():  # Check if any condition leads to a solution
                print(f">Invariant or overflow condition does not hold for first {k + 1} states.")
                for i, s in enumerate(states[:k + 1]):
                    print(f"> state {i}: x = {solver.get_value(s['x']).bv_unsigned_value()}, y = {solver.get_value(s['y']).bv_unsigned_value()}, "
                          f"a = {solver.get_value(s['a']).bv_unsigned_value()}, b = {solver.get_value(s['b']).bv_unsigned_value()}, "
                          f"z = {solver.get_value(s['z']).bv_unsigned_value()}, pc = {solver.get_value(s['pc']).bv_unsigned_value()}.")
                return  # Exit if invariant or overflow fails
            else:
                if k == K - 1:  # If we are at the last iteration and no issue was found
                    print(f">Invariant and overflow condition hold for the first {K} states.")
            solver.pop()  # Pop the last context off the stack to restore previous state

# Example with different random values
import random
for _ in range(3):  # Test with three different random (x, y) values
    x_val = random.randint(1, 2**(n // 2) - 1)  # Random value for x within limits
    y_val = random.randint(1, 2**(n // 2) - 1)  # Random value for y within limits
    print(f"Testing with x = {x_val}, y = {y_val}")
    bmc_always(invariant_check, 20, x_val, y_val)  # Run bounded model checking



Testing with x = 10, y = 23
>Invariant and overflow condition hold for the first 20 states.
Testing with x = 2, y = 17
>Invariant and overflow condition hold for the first 20 states.
Testing with x = 3, y = 3
>Invariant and overflow condition hold for the first 20 states.


# Restrictions for Solvers in Bounded Model Checking

When utilizing solvers like Z3 or PySMT for bounded model checking of the integer multiplication automaton, it’s important to adhere to the following restrictions:

1. **Bit-Vector Size**:
   - The bit-vector size (`n`) must be explicitly defined. In the provided code, `n` is set to 10, meaning all bit-vector variables will represent integers within the range \([0, 2^{10}-1]\).
   - Ensure all operations respect the limits imposed by the bit-vector size to avoid overflow and undefined behavior.

2. **Initial Values**:
   - The initial values for `a` and `b` should be constrained to less than \(2^{(n/2)}\). This is to ensure that during the multiplication process, the resultant value will not exceed the maximum representable value in `n` bits.
   - For example, if `n = 10`, both `a` and `b` should be less than \(32\).

3. **Invariant Conditions**:
   - The invariant \(x \times y + z = a \times b\) must be maintained throughout the state transitions. If this invariant does not hold at any state, it indicates a logical error in the multiplication implementation.
   - The model checker must regularly assert that this invariant holds true after each transition between states.

4. **Overflow Checks**:
   - The solver must check for potential overflow conditions for all state variables. If any variable exceeds \(2^n - 1\), it indicates an overflow that the program must handle.
   - Each state variable (`x`, `y`, `a`, `b`, `z`) should be validated against the maximum value allowed for their respective bit-vectors.

5. **Transition Validity**:
   - All state transitions must be defined correctly and should maintain the logical structure of the multiplication algorithm.
   - The transitions should ensure that the state machine can only move to valid next states according to the defined logic of multiplication.

6. **Bounded Model Checking Depth (K)**:
   - The depth \(K\) defines how many states the model checker will verify. A larger \(K\) can provide more extensive verification but at the cost of increased computational complexity.
   - Be mindful of the trade-off between the thoroughness of the checks and the performance of the solver.

7. **Solver Limitations**:
   - Different solvers may have limitations in terms of the size of the formulas they can handle. Always check the documentation for the specific limitations of the solver you are using.
   - Performance can vary based on the structure of the constraints, so consider optimizing your constraints for better solver performance.

Adhering to these restrictions is essential for ensuring the correctness and reliability of the model checking process when verifying the automaton that performs integer multiplication using bit-vectors.
